In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import accuracy_score, classification_report
from sklearn.inspection import permutation_importance
from scikeras.wrappers import KerasClassifier
from sklearn.utils.validation import check_X_y
from sklearn.impute import SimpleImputer

In [4]:
# Load and preprocess data
file_path = '/Users/pvuda/Documents/f/combined_file.csv'
data = pd.read_csv(file_path)

In [5]:
x_cols = ['spectral_bandwidth_min', 'spectral_contrast_mean',
       'spectral_contrast_std', 'spectral_contrast_var',
       'spectral_contrast_max', 'spectral_contrast_range',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis','spectral_contrast_mean', 'spectral_contrast_std',
       'spectral_contrast_var', 'spectral_contrast_max',
       'spectral_contrast_range', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_flatness_max','spectral_contrast_mean', 'spectral_contrast_std',
       'spectral_contrast_var', 'spectral_contrast_max',
       'spectral_contrast_range', 'spectral_contrast_skew',
       'spectral_flux_std', 'energy_std', 'energy_max', 'energy_range','spectral_bandwidth_25th_percentile', 'spectral_contrast_std',
       'spectral_contrast_var', 'spectral_contrast_75th_percentile',
       'spectral_contrast_skew', 'spectral_contrast_kurtosis', 'shimmer_local',
       'shimmer_apq3', 'shimmer_apq5', 'shimmer_dda','spectral_contrast_mean', 'spectral_contrast_max',
       'spectral_contrast_range', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'jitter_rap', 'jitter_ppq5','spectral_contrast_mean', 'spectral_contrast_std',
       'spectral_contrast_var', 'spectral_contrast_max',
       'spectral_contrast_range', 'spectral_contrast_skew',
       'spectral_flux_std', 'energy_std', 'energy_max', 'energy_range','spectral_contrast_mean', 'spectral_contrast_max',
       'spectral_contrast_range', 'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'jitter_ppq5', 'shimmer_local',
       'speakingrate', 'articulationrate', 'asd','spectral_contrast_mean', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'spectral_flatness_mean', 'jitter_local',
       'jitter_rap', 'jitter_ppq5','spectral_contrast_mean', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'jitter_rap', 'jitter_ppq5',
       'shimmer_apq3', 'shimmer_dda','spectral_contrast_std', 'spectral_contrast_var',
       'spectral_contrast_max', 'spectral_contrast_range',
       'spectral_contrast_25th_percentile', 'shimmer_apq3', 'shimmer_dda',
       'speakingrate', 'articulationrate', 'asd','spectral_contrast_mean', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'mfcc_4_75th_percentile', 'jitter_rap',
       'jitter_ppq5', 'hnr','spectral_contrast_mean', 'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'spectral_flux_mean', 'energy_mean',
       'energy_75th_percentile', 'jitter_rap', 'jitter_ppq5','spectral_contrast_mean', 'spectral_contrast_25th_percentile',
       'spectral_contrast_50th_percentile',
       'spectral_contrast_75th_percentile', 'spectral_contrast_skew',
       'spectral_contrast_kurtosis', 'shimmer_local', 'shimmer_apq3',
       'shimmer_apq5', 'shimmer_dda']
print(len(x_cols))

130


In [6]:
x_cols = list(set(x_cols))
print(len(x_cols))
print(x_cols)

33
['spectral_flatness_mean', 'speakingrate', 'spectral_contrast_max', 'spectral_contrast_std', 'shimmer_apq3', 'shimmer_apq5', 'energy_75th_percentile', 'spectral_bandwidth_25th_percentile', 'spectral_contrast_skew', 'jitter_ppq5', 'spectral_contrast_range', 'jitter_local', 'hnr', 'energy_range', 'articulationrate', 'spectral_contrast_50th_percentile', 'spectral_contrast_kurtosis', 'spectral_contrast_25th_percentile', 'spectral_bandwidth_min', 'energy_mean', 'spectral_contrast_mean', 'spectral_flatness_max', 'spectral_contrast_var', 'energy_std', 'jitter_rap', 'mfcc_4_75th_percentile', 'asd', 'shimmer_dda', 'spectral_flux_mean', 'spectral_flux_std', 'shimmer_local', 'energy_max', 'spectral_contrast_75th_percentile']


## Binary

In [5]:
# Create a new target variable for binary classification
data['real_or_fake_general'] = data['real_or_fake'].apply(lambda x: 'R' if x == 'R' else 'F')

# Encode the binary labels
label_encoder = LabelEncoder()
data['real_or_fake_general'] = label_encoder.fit_transform(data['real_or_fake_general'])

# Prepare feature matrix and target vector
# X = data.drop(columns=['audio_id', 'real_or_fake', 'real_or_fake_general'])
X = data[x_cols]
y = data['real_or_fake_general']

In [6]:
X.isna().sum()

spectral_contrast_75th_percentile      0
spectral_contrast_max                  0
spectral_bandwidth_25th_percentile     0
speakingrate                           1
spectral_contrast_kurtosis             0
energy_max                             0
jitter_ppq5                           25
shimmer_apq5                          89
spectral_contrast_25th_percentile      0
articulationrate                       1
asd                                   50
spectral_contrast_var                  0
jitter_rap                            15
spectral_flux_std                      0
shimmer_apq3                          22
spectral_contrast_skew                 0
energy_75th_percentile                 0
spectral_flux_mean                     0
spectral_flatness_max                  0
hnr                                    3
shimmer_dda                           22
spectral_bandwidth_min                 0
spectral_contrast_50th_percentile      0
spectral_contrast_range                0
spectral_contras

In [7]:
y.isna().sum()

0

In [8]:
# Handle missing values in X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

In [9]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [10]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# Reshape input data for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [14]:
# Define the LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], 1), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Wrap the Keras model with KerasClassifier
lstm_model = KerasClassifier(model=create_lstm_model, epochs=2, batch_size=32, verbose=1)

# Fit the model
history = lstm_model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
6811/6811 ━━━━━━━━━━━━━━━━━━━━ 74s 11ms/step - accuracy: 0.8547 - loss: 0.6373 - val_accuracy: 0.8761 - val_loss: 0.2631
Epoch 2/2
6811/6811 ━━━━━━━━━━━━━━━━━━━━ 72s 11ms/step - accuracy: 0.8810 - loss: 0.2593 - val_accuracy: 0.9129 - val_loss: 0.1990


In [14]:
# Make predictions
nn_predictions_prob = lstm_model.predict(X_test_reshaped)
nn_predictions = (nn_predictions_prob > 0.5).astype(int)

1703/1703 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step


In [15]:
# Evaluate the model
accuracy = accuracy_score(y_test, nn_predictions)
print("Neural Network Accuracy: ", accuracy)
print(classification_report(y_test, nn_predictions, target_names=label_encoder.classes_))

Neural Network Accuracy:  0.9129134089490878
              precision    recall  f1-score   support

           F       0.92      0.99      0.95     46897
           R       0.84      0.46      0.60      7589

    accuracy                           0.91     54486
   macro avg       0.88      0.73      0.77     54486
weighted avg       0.91      0.91      0.90     54486



In [16]:
# Custom scorer to handle reshaping
def custom_scorer(estimator, X, y):
    X_reshaped = X.reshape((X.shape[0], X.shape[1], 1))
    predictions = estimator.predict(X_reshaped)
    predictions = (predictions > 0.5).astype(int)
    return accuracy_score(y, predictions)

# Calculate permutation importance with custom scorer
result = permutation_importance(lstm_model, X_test, y_test, n_repeats=1, random_state=42, scoring=custom_scorer)

# Display feature importance
importance_df = pd.DataFrame(result.importances_mean, index=X.columns, columns=['Importance'])
print(importance_df)


1703/1703 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step


KeyboardInterrupt: 

In [ ]:
importance_df.sort_values

<bound method DataFrame.sort_values of                         Importance
spectral_centroid_mean         0.0
spectral_centroid_std          0.0
spectral_centroid_var          0.0
spectral_centroid_min          0.0
spectral_centroid_max          0.0
...                            ...
intensity_duration             0.0
speakingrate                   0.0
articulationrate               0.0
asd                            0.0
totalpauseduration             0.0

[408 rows x 1 columns]>

## Multi Label

In [7]:
# Encode the binary labels
label_encoder = LabelEncoder()
data['real_or_fake_general'] = label_encoder.fit_transform(data['real_or_fake'])

# Prepare feature matrix and target vector
X = data[x_cols]
y = data['real_or_fake_general']

In [8]:
y

0         6
1         3
2         2
3         5
4         1
         ..
272423    3
272424    0
272425    4
272426    1
272427    5
Name: real_or_fake_general, Length: 272428, dtype: int64

In [9]:
# Handle missing values in X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

In [10]:
# Optionally, scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
# Reshape input data for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [15]:
# Wrap the Keras model with KerasClassifier
lstm_model = KerasClassifier(model=create_lstm_model, epochs=2, batch_size=32, verbose=1)

# Fit the model
history = lstm_model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test))

Epoch 1/2


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6811/6811 ━━━━━━━━━━━━━━━━━━━━ 51s 7ms/step - accuracy: 0.1276 - loss: nan - val_accuracy: 0.1298 - val_loss: nan
Epoch 2/2
6811/6811 ━━━━━━━━━━━━━━━━━━━━ 64s 9ms/step - accuracy: 0.1297 - loss: nan - val_accuracy: 0.1298 - val_loss: nan


In [16]:
# Make predictions
nn_predictions_prob = lstm_model.predict(X_test_reshaped)
nn_predictions = (nn_predictions_prob > 0.5).astype(int)

1703/1703 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


In [17]:
# Evaluate the model
accuracy = accuracy_score(y_test, nn_predictions)
print("Neural Network Accuracy: ", accuracy)
print(classification_report(y_test, nn_predictions, target_names=label_encoder.classes_))

Neural Network Accuracy:  0.12984986969129686
              precision    recall  f1-score   support

         F01       0.13      1.00      0.23      7075
         F02       0.00      0.00      0.00      7091
         F03       0.00      0.00      0.00      7110
         F04       0.00      0.00      0.00      6961
         F05       0.00      0.00      0.00      7061
         F06       0.00      0.00      0.00      6954
           R       0.00      0.00      0.00      7589
         WF1       0.00      0.00      0.00       652
         WF2       0.00      0.00      0.00       695
         WF3       0.00      0.00      0.00       638
         WF4       0.00      0.00      0.00       656
         WF5       0.00      0.00      0.00       651
         WF6       0.00      0.00      0.00       683
         WF7       0.00      0.00      0.00       670

    accuracy                           0.13     54486
   macro avg       0.01      0.07      0.02     54486
weighted avg       0.02      0.13 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  